In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import wandb
import tempfile
import os
from models.normalizing_flow import HierarchicalNormalizingFlowSB
import numpy as np
import torch
import matplotlib.pyplot as plt
from torch import nn


In [3]:
from train_flow_sb import unflatten, object_from_config

In [4]:
api = wandb.Api()

In [5]:
run = api.run('druhe/gw-src/16pgb80c')

In [6]:
config = unflatten(run.config)

In [7]:
dataset = object_from_config(config, key='dataset')(**config['dataset'])

In [8]:
sb_weight = config['model']['sb_weight']

In [9]:
flows = object_from_config(config, key='flow')(**config['flow'])

In [10]:
checkpoints = [f for f in run.files() if f.name.endswith('.ckpt')]

In [11]:
tempdir = tempfile.TemporaryDirectory()

In [12]:
ckpt = checkpoints[0]

In [13]:
ckpt_path = ckpt.download(root=tempdir.name, replace=True)

In [14]:
model = HierarchicalNormalizingFlowSB.load_from_checkpoint(ckpt_path.name, dataset=dataset, sb_weight=sb_weight, flows=flows)

INFO:torch.distributed.nn.jit.instantiator:Created a temporary directory at /var/folders/d3/x840qlg17x1f92cnsrkq62fw0000gn/T/tmper60dl8g
INFO:torch.distributed.nn.jit.instantiator:Writing /var/folders/d3/x840qlg17x1f92cnsrkq62fw0000gn/T/tmper60dl8g/_remote_module_non_sriptable.py


In [15]:
# with torch.no_grad():
#     axes_names = []
#     axes = []

#     for n, ax in dataset.grid.items():
#         axes_names.append(n)
#         axes.append(ax)

#     m1, m2, z = np.stack(np.meshgrid(*axes, indexing="xy")).reshape(3, -1)

#     if dataset.has_normalization:
#         raise NotImplementedError
#         x, y = dataset.normalize_forward(x, y)

#     resolutions = [len(ax) for ax in axes]

#     input = np.stack([m1, m2, z], axis=-1)
#     input = torch.from_numpy(input).float()
#     prob = model.log_prob(input).exp().view(*resolutions)


#     pm1m2 = prob.sum(-1)

#     fig = plt.figure(figsize=(16, 16), facecolor="white")
#     plt.imshow(
#         pm1m2,
#         cmap="jet",
#         origin="lower",
#         extent=(
#             axes[0][0],
#             axes[0][-1],
#             axes[1][0],
#             axes[1][-1],
#         ),  # origin='lower' changes the order
#         aspect="auto",
#     )

#     plt.xlabel(axes_names[0])  # origin='lower' changes the order
#     plt.ylabel(axes_names[1])
#     plt.tight_layout()



In [16]:
axes_names = []
axes = []

for n, ax in dataset.grid.items():
    axes_names.append(n)
    axes.append(ax)

m1, m2, z = np.stack(np.meshgrid(*axes, indexing="xy")).reshape(3, -1)

if dataset.has_normalization:
    raise NotImplementedError
    x, y = dataset.normalize_forward(x, y)

resolutions = [len(ax) for ax in axes]

input = np.stack([m1, m2, z], axis=-1)


In [17]:
input=torch.from_numpy(input).double()

In [18]:
import hamiltorch

In [19]:
from torch.nn.utils import _stateless


In [29]:
def flatten_params(parameters):
    """
    flattens all parameters into a single column vector. Returns the dictionary to recover them
    :param: parameters: a generator or list of all the parameters
    :return: a dictionary: {"params": [#params, 1],
    "indices": [(start index, end index) for each param] **Note end index in uninclusive**

    """
    l = [torch.flatten(p) for p in parameters]
    indices = []
    s = 0
    for p in l:
        size = p.shape[0]
        indices.append((s, s+size))
        s += size
    flat = torch.cat(l).view(-1, 1)
    return flat, indices


def unflatten_params(flat_params, indices, model):
    """
    Gives a list of recovered parameters from their flattened form
    :param flat_params: [#params, 1]
    :param indices: a list detaling the start and end index of each param [(start, end) for param]
    :param model: the model that gives the params with correct shapes
    :return: the params, reshaped to the ones in the model, with the same order as those in the model
    """
    l = [flat_params[s:e] for (s, e) in indices]
    for i, p in enumerate(model.parameters()):
        l[i] = l[i].view(*p.shape)
    return tuple(nn.Parameter(p) for p in l)


In [30]:
params_flat, indices = flatten_params(model.parameters())
names = list(n for n, _ in model.named_parameters())


params_flat = params_flat.squeeze()

In [31]:
model=model.double()

params_init = hamiltorch.util.flatten(model).clone().double()


In [32]:
fmodel = hamiltorch.util.make_functional(model)

In [36]:
def log_prob(params, y=None):
    prior = torch.distributions.Normal(params_init, 1).log_prob(params)

    params = unflatten_params(params, indices, model)
            
    out: torch.Tensor = _stateless.functional_call(model, {n: p for n, p in zip(names, params)}, input)    
    ll = prior.sum() + out.sum()
    print(out.mean().item())
    return ll




In [37]:
N = 300
burn = 200
N_nuts = N + burn
step_size = 1e-2
L = 20

hamiltorch.sample(
    log_prob, 
    params_init=params_init, 
    num_samples=N_nuts, num_steps_per_sample
    =L, step_size=step_size, burn=burn, )







Sampling (Sampler.HMC; Integrator.IMPLICIT)
Time spent  | Time remain.| Progress             | Samples | Samples/sec
-258.8183631382832
-258.8183631382832
-791.3876175427846
-563.0427895238288
-714.6202286248185
-937.2559735191967
-1149.1823304826748
-1332.1297743550433
-1485.48737085598
-1613.493071029463
-1720.93595486402
-1811.9521560357257
-1889.8497029088917
-1957.2202475136783
-2016.0960715990802
-2068.0808311168594
-2114.440930564363
-2156.1706664319313
-2194.038458974127
-2228.6411205655004
-2260.456291642062
-2289.8808815953835
-2289.8808815953835
-258.81836313828320:08:40 | -------------------- |   1/500 | 0.96       
-258.8183631382832
-468.0544499209573
-853.4726915896724
-1168.6509631114454
-1408.3042942586594
-1585.1137446569776
-1715.206168247502
-1810.6708600215493
-1878.9564447580783
-1923.535226566257
-1943.6650130090645
-1931.9247107470314
-1867.819345695304
-1745.1162584390318
-2331.9326613940248
-2583.0830627491214
-2656.8631416342237
-2744.539606301988
-2824.06091

-1630.1615369910946
-5959.394201837327
-618.9254166889682
-943.6850270727163
-1362.0169184254423
-1591.635161273359
-1759.3277780847775
-1916.7387184282834
-2096.2227352584064
-2309.3434330514447
-2599.591368081178
-2873.5982615001067
-3079.8389222513933
-3242.211978627185
-3355.246178509622
-3435.9627746194665
-3485.974404093552
-3518.893449795649
-3549.0786430730655
-3549.0786430730655
-258.81836313828320:17:52 | #------------------- |  17/500 | 0.45       
-258.8183631382832
-1886.1741439928949
-1529.2442435518085
-1824.8678254448482
-2104.922093692722
-2322.6385332311334
-2486.0968447059668
-2609.872480454541
-2705.6524493757624
-2781.7408757569583
-2844.34415308605
-2899.167202177381
-2949.52511028711
-2995.5119186154398
-3037.43023011239
-3076.224055110789
-3112.9422642630498
-3148.2809109427258
-3182.4706886746135
-3215.521154052868
-3247.4459660723232
-3247.4459660723232
-258.81836313828320:17:50 | #------------------- |  18/500 | 0.45       
-258.8183631382832
-742.94728427132

-2537.536231043605
-2614.5016823252145
-2657.9570745854494
-2684.2234581958173
-2701.0835388894466
-2712.5628651308107
-2720.9627945612506
-2720.9627945612506
-258.81836313828320:16:48 | #------------------- |  33/500 | 0.46       
-258.8183631382832
-390.99867268643345
-649.6518339072813
-1028.619580502932
-1286.404923783341
-1446.6377413704722
-1566.7777392874195
-1663.769542020375
-1742.0204506893092
-1807.4038773353427
-1866.1966260892698
-1924.0620481714955
-1986.0950503288307
-2057.014497942445
-2141.193783971529
-2237.827940747456
-2327.825324567794
-2403.148718044859
-2466.6137217959204
-2520.873884504903
-2567.910848462153
-2567.910848462153
-258.81836313828320:16:44 | #------------------- |  34/500 | 0.46       
-258.8183631382832
-478.1385870899536
-1060.7506112337194
-1440.1144508128814
-1695.5917492877988
-1883.4926052834892
-2027.6401621382329
-2140.239339975458
-2228.9141792167657
-2299.0736021505704
-2354.8224548569888
-2399.380133523017
-2435.3107442039927
-2464.640533

-258.81836313828320:15:47 | ##------------------ |  49/500 | 0.48       
-258.8183631382832
-254.4659940764309
-467.9707100071053
-700.3741720419758
-852.7211471275289
-965.6567131120667
-1058.9542925833402
-1139.4432546524472
-1210.3189933699346
-1273.6560312080323
-1331.033481867879
-1383.7131426399228
-1432.711999399111
-1478.849301304483
-1522.7860992856906
-1565.0585589164111
-1606.1053383849735
-1646.288797439191
-1685.9112099945842
-1725.2271125900813
-1764.4529844243261
-1764.4529844243261
-258.81836313828320:15:45 | ##------------------ |  50/500 | 0.48       
-258.8183631382832
-1326.9555172260887
-1433.9367297173949
-1631.7653548105977
-1851.0061960048208
-2036.7373652369547
-2190.409583640634
-2320.056175530861
-2433.5834328985766
-2537.7766651725574
-2638.193036052164
-2739.3571542791296
-2844.86458820857
-2956.1836022098905
-3068.577932373992
-3175.4385160242255
-3275.1259090532794
-3367.853303721124
-3454.0561172333387
-3534.178539668048
-3608.6521453730375
-3608.6521453

-837.1728195879961
-1072.7254531513142
-1272.9922571948769
-1439.3149300568696
-1574.215292872874
-1680.7003079648666
-1763.8768871757015
-1828.282072141773
-1877.198953446425
-1913.0671426710167
-1937.8906420841508
-1953.512990884844
-1961.7787672320896
-1964.5963409183378
-1963.9166912020614
-1961.653066821031
-1959.5752349723625
-1959.2125454358936
-1959.2125454358936
-258.81836313828320:15:10 | ###----------------- |  66/500 | 0.48       
-258.8183631382832
-402.412071520481
-730.6199682355292
-1034.359626707023
-1281.8533694497871
-1478.0230423903777
-1631.859860604479
-1753.0893574398551
-1850.1311234434384
-1929.0067189466315
-1993.9741390545464
-2048.1189805825375
-2093.728750527373
-2132.53076772918
-2165.849269225868
-2194.712638832848
-2219.928994664638
-2242.140797979827
-2261.8628851398553
-2279.4999199584245
-2295.378087274357
-2295.378087274357
-258.81836313828320:15:10 | ###----------------- |  67/500 | 0.48       
-258.8183631382832
-361.88481412663117
-1201.0841174623

...

-258.81836313828320:02:08 | ##################-- | 441/500 | 0.46       
-258.8183631382832
-412.2254921824292
-619.1496076407795
-861.2082056527852
-1087.2459590726949
-1278.3655142187285
-1436.333237847823
-1567.1866258155183
-1676.692855359494
-1769.477944870728
-1849.0783639124536
-1918.1687274597498
-1978.7797302052666
-2032.4660689828397
-2080.433236371239
-2123.6257924075435
-2162.7938540532577
-2198.5386935661904
-2231.347448544198
-2261.6199797504505
-2289.6858682679162
-2289.6858682679162
-258.81836313828320:02:05 | ##################-- | 442/500 | 0.46       
-258.8183631382832
-446.8334229797895
-1051.4680527884673
-1583.814405384352
-1892.4277514949902
-2052.415731302269
-2146.004445423071
-2213.654641800592
-2270.588262063342
-2323.17238664851
-2376.411729618322
-2436.223932303137
-2507.4820502231764
-2594.703882129189
-2699.1695433832365
-2813.178262056643
-2925.545130654894
-3032.212950506706
-3133.1351445513137
-3228.782472917259
-3320.307217492282
-3320.307217492282
-

-2606.669879353877
-2786.976470138864
-2914.603226317164
-3007.6499192669617
-3076.8360431723067
-3128.734918313956
-3167.4850289321193
-3195.736714011614
-3215.1865986306643
-3226.882663365521
-3231.3939435045413
-3228.892362931002
-3219.1685797593327
-3201.587619561253
-3174.9790515519444
-3137.4546906436226
-3137.4546906436226
-258.81836313828320:01:31 | ##################-- | 458/500 | 0.46       
-258.8183631382832
-806.3310564597335
-1083.365347325352
-1381.7830469287417
-1608.4223929334644
-1781.8199865421243
-1920.3988185610785
-2032.9334437209104
-2127.280510853636
-2209.531841744472
-2283.785992255289
-2352.97403706563
-2419.8688519454054
-2488.3768952591754
-2561.94897819327
-2634.9803390345674
-2703.2745901315084
-2768.767533808425
-2832.9096670049776
-2896.291772676077
-2959.1798098648187
-2959.1798098648187
-258.81836313828320:01:28 | ##################-- | 459/500 | 0.46       
-258.8183631382832
-349.70741215556563
-432.21519074443216
-1302.860889760031
-1579.0477898236

-2120.8501085037033
-2203.8219390832687
-2284.480094711314
-2362.046255777316
-2438.9510179359877
-2525.8909212732287
-2638.5755813900323
-2638.5755813900323
-258.81836313828320:00:56 | ###################- | 474/500 | 0.46       
-258.8183631382832
-414.0445794848773
-1063.7261044249285
-1301.8534645838868
-1611.923584176243
-1888.7482684164409
-2083.4566152564134
-2217.397899759273
-2313.111871429848
-2385.3909989702724
-2442.8621104778776
-2490.447982995833
-2531.0098355223363
-2566.2653296398553
-2597.282882526934
-2624.7545555672973
-2649.1504492481236
-2670.8078012338974
-2689.9839907102682
-2706.8883230184642
-2721.7008177927505
-2721.7008177927505
-258.81836313828320:00:54 | ###################- | 475/500 | 0.46       
-258.8183631382832
-254.98751708278485
-590.4071410677236
-1004.5397860984656
-1228.3592005918858
-1342.0918286446683
-1425.9751105423784
-1502.3700469863045
-1574.9048530951336
-1642.793679651103
-1704.8552426963583
-1760.466429388102
-1809.6054212026363
-1852.6

-258.81836313828320:00:21 | #################### | 490/500 | 0.46       
-258.8183631382832
-635.2423192184299
-707.0917982492713
-833.2468003870474
-1065.8681278616627
-1255.8425589174594
-1405.8937980838882
-1525.8458195423987
-1623.6337286204973
-1705.0945914294261
-1774.5211100320626
-1835.1438028206594
-1889.4743647479545
-1939.5430007821542
-1987.0673890964745
-2033.584452999287
-2080.5724133297645
-2129.586179847229
-2182.388212166864
-2240.846896878711
-2305.892010194058
-2305.892010194058
-258.81836313828320:00:19 | #################### | 491/500 | 0.46       
-258.8183631382832
-409.3755096889922
-789.568367825078
-1202.60945288711
-1609.5871455208799
-1892.2253333784024
-2074.690617138085
-2207.932427324469
-2322.0094001557645
-2432.9899595256834
-2552.0402509136225
-2689.432958138655
-2848.012086381635
-3013.084653912765
-3167.706319685869
-3308.849944537894
-3436.880955068891
-3551.663986036273
-3653.4475546986787
-3743.26993244321
-3822.671610571763
-3822.671610571763
-25

[tensor([-2.3550e+00, -1.9355e+00, -3.6661e+00,  2.3216e-02,  2.0492e-02,
         -4.7562e-02,  2.8246e+00,  1.7371e+00, -4.1685e-03, -1.1484e-03,
          4.0934e-04, -1.8839e+00, -2.3366e+00,  2.7113e-03,  4.9928e-03,
          3.6793e-02, -3.1709e+00, -2.5666e+00,  4.9370e-03,  3.9433e-03,
          1.1853e-01, -5.5081e+00, -1.8446e+00,  6.2483e-03,  4.8495e-03,
         -1.5682e-01,  5.0755e+00,  3.2241e+00, -4.9285e-03, -1.3481e-02,
         -4.0563e-02,  3.8434e+00,  9.2220e+00, -8.4619e-02, -2.3256e-01,
          2.1243e-01,  1.6690e+01, -3.0715e+00, -1.8416e-01,  1.7021e-02],
        dtype=torch.float64),
 tensor([-2.3550e+00, -1.9355e+00, -3.6661e+00,  2.3216e-02,  2.0492e-02,
         -4.7562e-02,  2.8246e+00,  1.7371e+00, -4.1685e-03, -1.1484e-03,
          4.0934e-04, -1.8839e+00, -2.3366e+00,  2.7113e-03,  4.9928e-03,
          3.6793e-02, -3.1709e+00, -2.5666e+00,  4.9370e-03,  3.9433e-03,
          1.1853e-01, -5.5081e+00, -1.8446e+00,  6.2483e-03,  4.8495e-03,
       

In [ ]:
hamiltorch.sample(log_prob_func, params_init, step_size=step_size)


In [ ]:
import hamiltorch
L = 5
step_size = .3
N = 2048

hamiltorch.set_random_seed(123)
params_init = params_flat
burn=500
N_nuts = burn + N
params_hmc_nuts = hamiltorch.sample(log_prob_func=log_prob, params_init=params_init,
                                                  num_samples=N_nuts,step_size=step_size,num_steps_per_sample=L,
                                                  sampler=hamiltorch.Sampler.HMC_NUTS, burn=burn,
                                                  desired_accept_rate=0.8)


In [ ]:
def log_prob(x):
    x = x[None]
    return model.log_prob(x).sum()

In [ ]:
v, _ = input.median(dim=0)

In [ ]:
log_prob(v)

In [ ]:
L = 5
step_size = .3
N = 2048

hamiltorch.set_random_seed(123)
params_init = v
burn=500
N_nuts = burn + N
params_hmc_nuts = hamiltorch.sample(log_prob_func=log_prob, params_init=params_init,
                                                  num_samples=N_nuts,step_size=step_size,num_steps_per_sample=L,
                                                  sampler=hamiltorch.Sampler.HMC_NUTS, burn=burn,
                                                  desired_accept_rate=0.8)


In [ ]:
import seaborn as sns

In [ ]:
samples = torch.stack(params_hmc_nuts)